# SNLP Assignment 3

Name 1: Rayyan Mohammad Minhaj

Student id 1: 7074982

Email 1:rami00002@stud.uni-saarland.de

Name 2: Abdullah Abdul Wahid

Student id 2: 7075730

Email 2:  abyy00002@stud.uni-saarland.de

Instructions: Read each question carefully. 

Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the respective Python files for any additional exercises necessary. There is no need to submit the data files should they exist. 

Upload the zipped folder on CMS. Please follow the naming convention of Name1_studentID1_Name2_studentID2.zip. Make sure to click on "Turn-in" (or the equivalent on CMS) after your upload your submission, otherwise the assignment will not be considered as submitted. Only one member of the group should make the submisssion.

In [1]:
! pip install pandas==1.3.5

     -------------------------------------- 10.2/10.2 MB 129.6 kB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3


You should consider upgrading via the 'C:\Users\rayya\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import string
import pandas as pd
from math import log2
from typing import List, Dict
from collections import Counter

## Exercise 1: Cross-Entropy and KL-Divergence (6 points)

### Theory

Recall the formulas for Cross-Entropy:

$$H(P, Q) = -\sum_{x \in X} P(x) \times \log{Q(x)}$$

And KL-Divergence:

$$D_{KL}(P || Q) = \sum_{x \in X} P(x) \times \log{\frac{P(x)}{Q(x)}}$$

where P often signifies the empirical or observed probability distribution and Q signifies the estimated distribution.


1. A common way to train a language model is to minimize the Cross-Entropy score. Explain why minimizing Cross-Entropy is equivalent to minimizing KL-Divergence. Support your answer with a mathematical expression. [1 point] <br/>




2. For a function $d$ to be considered a distance metric, the following three properties must hold:

    $\forall x,y,z \in U:$

    1. $d(x,y) = 0 \Leftrightarrow x = y$
    2. $d(x,y) = d(y,x)$
    3. $d(x,z) \le d(x,y) + d(y,z)$

    Is $D_{KL}$ a distance metric? ($U$ in this case is the set of all distributions over the same possible states).
For each of the three points either prove that it holds for KDL​ or show a counterexample why not. [2 points]

## Answers

1 - While training a model, we want Q (the predictions) to get as close as possible to P (ground truth). We also know that Entropy (H) measures the uncertainty of truth, cross-entropy measures the avg bits needed to encode samples from P to Q, and KL-Divergence measures the how much difference there is between model and actual truth. So we can mathematically say: <br/>

$H(P,Q) = H(P) + D_{KL}(P||Q)$ <br/>

Thus, minimizing Cross-Entropy is equivalent to minimizing KL-Divergence because the only part that depends on the model’s predictions is the KL part. The other part (entropy of the real data - H(P)) is fixed.

<br/>

2- For the first property, yes it holds and we can check this with a simple calculation:<br/>
$x=y=3$ <br/>
$D_{KL}(x||y) = 3 * log(3/3) = 0$ <br/>
This is mainly because log(1) is equal to 0. <br/>

<br/>
For second property, it does not hold as KL-Divergence is not symmetric. We can verify this using a simple counter example: <br/>

$x=P=3$ and $y=Q=4$ <br/>
$D_{KL}(x||y) = 3 * log(3/4) = -0.86$ <br/>
$D_{KL}(y||x) = 4 * log(4/3) = 1.15$ <br/>

<br/>
For last property, it also does not hold, again we can verify this using a simple counter example: <br/>

$x=3$ and $y=4$ and $z=5$<br/>
$D_{KL}(x||z) = 3 * log(3/5) = -1.53$ <br/>
$D_{KL}(x||y) = 3 * log(3/4) = -0.86$ <br/>
$D_{KL}(y||z) = 4 * log(4/5) = -0.89$ <br/>

$-1.53 <= -1.75$ <br/>
therefore, we can say KL-Divergence is not a distance measure.




### Code

Suppose we have three coins. Here are the results of flipping each coin 20 times:

In [2]:
coin1 = "H T H T H T H T H T H T H T H T H T H T"
coin2 = "H H H H T H H T H H H T T H H T H H H H"
coin3 = "T T T T T T T T T T T T T H T T T T H T"

Let's turn the sequences into lists of tokens:

In [3]:
def tokenize(s : str) -> List[str]:
    return s.split()

coin1_tokens = tokenize(coin1)
coin2_tokens = tokenize(coin2)
coin3_tokens = tokenize(coin3)

In [5]:
print(coin1_tokens)

['H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T', 'H', 'T']


Write the methods for a unigram model that
1. Estimate a probability distribution, given the tokenized text (use the imported `Counter`). Make sure that it is possible to update the model's distribution. [0.5 points]
2. Calculate the cross entropy between the model's estimated distribution and some given probability distribution. [1 point]
3. Calculate the KL-Divergence. [1 point]

**NOTE**: So far, we haven't covered strategies for dealing with out-of-vocabulary tokens. For now, we will accept if you:
 * Include only the tokens that are present in both distributions when calculating Cross-Entropy and KL-Divergence, i.e. ignore the tokens that don't appear in both distributions.
 * Normalize the resulting distributions so that they sum up to one.

Feel free to write separate methods for those functions.

In [76]:
class UnigramModel:
    
    def __init__(self):
        self.dist = {}
        self.freq = {}
    
    def fit(self, data: List[str]) -> Dict[str, float]:
        """Define a probability distribution for the model
        and assign it to self.dist
        
        Args:
            data - list of tokens
        """
        self.freq = Counter(data) #this is just simple freq of heads and tails

        count = Counter(data)
        total = sum(count.values())

        for token, count in count.items():
            self.dist[token] = count/total 

    def cross_entropy(self, distribution: Dict[str, float]) -> float:
        """Calculate the Cross-Entropy between the model's and a given distribution
        
        Args:
            distribution - dictionary of token probabilities
        Returns:
            cross_entropy - the Cross-Entropy value
        """
        # my P and Q look like: {'H':0.5, 'T':0.5}
        #H(P,Q) = - SUM -> P(x) * log(Q(x)) ------ here P(x) is basically head, tails self.dist and Q(x) is the distribution

        cross_entropy = 0

        for i in range(0, len(self.dist.items())):
            #print(f'P({i}): {list(self.dist.values())[i]} and Q({i}): {list(distribution.values())[i]}')
            cross_entropy +=list(self.dist.values())[i] * log2(list(distribution.values())[i])
        
        cross_entropy = -(cross_entropy)

        return cross_entropy



         
    
    def kl_divergence(self, distribution: Dict[str, float]) -> float:
        """Calculate the KL divergence between the model's and a given distribution

        Args:
            distribution - dictionary of token probabilities
        Returns:
            kl_divergence - the KL-Divergence value
        """
        #KLD (P||Q) = SUM-> P(x) * log(P(x)/Q(x))

        kl_divergence = 0

        for i in range(0, len(self.dist.items())):
            #print(f'P({i}): {list(self.dist.values())[i]} and Q({i}): {list(distribution.values())[i]}')
            kl_divergence +=list(self.dist.values())[i] * log2(list(self.dist.values())[i]/list(distribution.values())[i])
        

        return kl_divergence


Now fit the models on the provided coins and print out the distributions.

In [77]:
coin_model1 = UnigramModel()
coin_model2 = UnigramModel()
coin_model3 = UnigramModel()

coin_model1.fit(coin1_tokens)
coin_model2.fit(coin2_tokens)
coin_model3.fit(coin3_tokens)

print("Model 1 estimated probabilities")
for token, prob in sorted(coin_model1.dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{token}: {prob:.4f}")

print()
print("Model 2 estimated probabilities")
for token, prob in sorted(coin_model2.dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{token}: {prob:.4f}")

print()
print("Model 3 estimated probabilities")
for token, prob in sorted(coin_model3.dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{token}: {prob:.4f}")

Model 1 estimated probabilities
H: 0.5000
T: 0.5000

Model 2 estimated probabilities
H: 0.7500
T: 0.2500

Model 3 estimated probabilities
T: 0.9000
H: 0.1000


Update Model 2 with some additional data:

In [78]:
coin2_ext = "H H T H H H T T H H H H H H H H H T T H H H H H H T H H H T H H H"
coin2_ext_tokens = tokenize(coin2_ext)
coin_model2.fit(coin2_ext_tokens)

print("Model 2 updated estimated probabilities")
for token, prob in sorted(coin_model2.dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{token}: {prob:.4f}")

Model 2 updated estimated probabilities
H: 0.7879
T: 0.2121


Let's assume the empirical probability distribution for all coins is actually uniform. Calculate the Cross-Entropy:

In [79]:
uniform_coin_dist = {
    "H": 0.5,
    "T": 0.5
}

print("Cross-Entropy with Uniform Distribution")

print("Model 1 Cross-Entropy:", coin_model1.cross_entropy(uniform_coin_dist))
print("Model 2 Cross-Entropy:", coin_model2.cross_entropy(uniform_coin_dist))
print("Model 3 Cross-Entropy:", coin_model3.cross_entropy(uniform_coin_dist))

Cross-Entropy with Uniform Distribution
Model 1 Cross-Entropy: 1.0
Model 2 Cross-Entropy: 1.0
Model 3 Cross-Entropy: 1.0


Try it out with another distribution of your choosing:

In [80]:
coin_dist = {
    "H": 0.1,
    "T": 0.9
}

print("Cross-Entropy with Distribution:")
print("Model 1 Cross-Entropy:", coin_model1.cross_entropy(coin_dist))
print("Model 2 Cross-Entropy:", coin_model2.cross_entropy(coin_dist))
print("Model 3 Cross-Entropy:", coin_model3.cross_entropy(coin_dist))

Cross-Entropy with Distribution:
Model 1 Cross-Entropy: 1.736965594166206
Model 2 Cross-Entropy: 2.649519761248084
Model 3 Cross-Entropy: 3.004935594743131


Calculate KL-Divergence:

In [81]:
print("KL-Divergence with Uniform Distribution:")
print("Model 1 KL Divergence:", coin_model1.kl_divergence(uniform_coin_dist))
print("Model 2 KL Divergence:", coin_model2.kl_divergence(uniform_coin_dist))
print("Model 3 KL Divergence:", coin_model3.kl_divergence(uniform_coin_dist))

KL-Divergence with Uniform Distribution:
Model 1 KL Divergence: 0.0
Model 2 KL Divergence: 0.25448215718917155
Model 3 KL Divergence: 0.5310044064107189


In the `data` folder you are provided with:

* `unigram_freq.csv`: a file containing information on ~300k top words and their counts, derived from the Google Web Trillion Word Corpus (comma-separated).
* `frankenstein.txt`: the novel "Frankenstein" by Mary Shelly.
* `wikipedia.txt`: English Wikipedia corpus.
* `code.txt`: A small corpus of code, taken from the [codeparrot/github-code](https://huggingface.co/datasets/codeparrot/github-code) dataset.

To load and tokenize the texts, feel free to reuse the functions you wrote in your first assignment.

In [82]:
def load_text(filepath: str = 'data.txt') -> str:
    """
    Load text from a file.

    Args:
        filepath: Path to the file to be loaded.
    Returns:
        The content of the file as a string.
    """
    with open(filepath, 'r', encoding='utf-8') as f:
            contents= f.read()
    
    return contents

def preprocess(text: str) -> List[str]:
    """
    Preprocess the input text by lowercasing and removing punctuation.

    Args:
        text: The input text to preprocess.

    Returns:
        list: A list of tokens after preprocessing.
    """
    text = text.lower()
    cleaned_text = ''.join(char for char in text if char not in string.punctuation)
    tokens = cleaned_text.split()
    return tokens

Load the `unigram_freq.csv` data (use `pandas`) and create a dictionary representing a probability distribution (word: probability). Put the dictionary into a variable `empirical_dist`. [0.5 points]

For the sake of this exercise, we will assume it as the true probability distribution of American English and that no other words appear in the distribution apart from those listed in the dataset.

In [99]:
df = pd.read_csv('data/unigram_freq.csv')
print(df.head())

total = 0
samp_dict = {}

for i in range(0, len(df)):
    total += df.iloc[i,1]

for i in range(0, len(df)):
    samp_dict[df.iloc[i,0]] = df.iloc[i,1]/total

empirical_dist =  samp_dict

  word        count
0  the  23135851162
1   of  13151942776
2  and  12997637966
3   to  12136980858
4    a   9081174698


In [93]:
print(total)

588124220187


In [101]:
#printing the first 20 here because printing the entire dict was taking too long :(
from itertools import islice

for key, value in islice(empirical_dist.items(), 20):
    print(f"{key}: {value}")

the: 0.03933837507090547
of: 0.022362525338300483
and: 0.022100157619537028
to: 0.020636764209678228
a: 0.015440912627459126
in: 0.014400707674149294
for: 0.010088551882990708
is: 0.008001275333472512
on: 0.006376923565241907
that: 0.005781144503654221
by: 0.005696158661744654
this: 0.005489435157037871
with: 0.0054123101306521575
i: 0.0052475738476111455
you: 0.005094469709217781
it: 0.004783281792247097
not: 0.00447777365836533
or: 0.004405089635955221
be: 0.004078601220057391
are: 0.00406991378324621


Calculate the following Cross-Entropy and KL-Divergence Scores.

In [102]:
text_frankenstein = load_text('data/frankenstein.txt')
tokens_frankenstein = preprocess(text_frankenstein)

model_frankenstein = UnigramModel()
model_frankenstein.fit(tokens_frankenstein)

print("Frankenstein Cross-Entropy with Empirical Distribution:", model_frankenstein.cross_entropy(empirical_dist))
print("Frankenstein KL Divergence with Empirical Distribution:", model_frankenstein.kl_divergence(empirical_dist))

Frankenstein Cross-Entropy with Empirical Distribution: 11.424272427328797
Frankenstein KL Divergence with Empirical Distribution: 1.9832825634349243


In [103]:
text_wikipedia = load_text('data/wikipedia.txt')
tokens_wikipedia = preprocess(text_wikipedia)

model_wikipedia = UnigramModel()
model_wikipedia.fit(tokens_wikipedia)
print("Wikipedia Cross-Entropy with Empirical Distribution:", model_wikipedia.cross_entropy(empirical_dist))
print("Wikipedia KL Divergence with Empirical Distribution:", model_wikipedia.kl_divergence(empirical_dist))

Wikipedia Cross-Entropy with Empirical Distribution: 13.55586777034593
Wikipedia KL Divergence with Empirical Distribution: 1.9832013262654153


In [104]:
text_code = load_text('data/code.txt')
tokens_code = preprocess(text_code)

model_code = UnigramModel()
model_code.fit(tokens_code)
print("Code Cross-Entropy with Empirical Distribution:", model_code.cross_entropy(empirical_dist))
print("Code KL Divergence with Empirical Distribution:", model_code.kl_divergence(empirical_dist))

Code Cross-Entropy with Empirical Distribution: 15.669226023405667
Code KL Divergence with Empirical Distribution: 2.9864689518795524


## Text Compression (4 points)

Let's say we want to compress our datasets with a prefix-free binary code.

1. Write a function that computes the optimal length for each word in a given distribution. [0.5 points]
<!-- As explained in the lecture, a nice way of constructing a code would be, is to determine the length of the encoding a token based on the frequency of the token. This can be done in many ways. In the lecture we talked about prefix codes:
No code word is a prefix of another code wordWe can organize the code as a tree

Given an arbitrary alphabet along with probabilities for each token, you are to implement a function that outputs the encoding for each character. (3 points.)

**HINT**: feel free to use the example in the slides to validate that your generated encoding is correct:


| word | frequency | $C(\text{word})$ |
|:-----|:----------|:-----------------|
| the  |0.5        |`0`               |
| and  |0.25       |`10`              |
| of   |0.125      |`110`             |
| he   |0.125      |`111`             |


Where $C(\text{word})$ represents the encoding of word.

Though this algorithm is generalizable to any base of the code (i.e. the code need not be binary), we shall limit this exercise to binary encoding. -->

In [ ]:
#FORMULA IS IN SLIDE 35 of LECTURE 4
from math import ceil

def optimal_binary_length(distribution : Dict[str, float]) -> Dict[str, int]:
    """
    Calculate the optimal binary length for a given distribution.

    Args:
        distribution: A dictionary of token probabilities.

    Returns:
        A dictionary with tokens as keys and their optimal binary lengths as values.
    """
    # optimal len = ciel(-log2(p)) --- log2 here because theyre asking for binary len
    optimal_dict = {}
    for key, value in distribution.items():
        optimal_dict[key] = ceil(-log2(value))
    
    return optimal_dict


In [108]:
print("Optimal Encoding Length: \"if\"")
print(f"Frankenstein:\t{optimal_binary_length(model_frankenstein.dist)['if']:.4f}")
print(f"Wikipedia:\t{optimal_binary_length(model_wikipedia.dist)['if']:.4f}")
print(f"Code:\t\t{optimal_binary_length(model_code.dist)['if']:.4f}")
print()
print("Optimal Encoding Length: \"the\"")
print(f"Frankenstein:\t{optimal_binary_length(model_frankenstein.dist)['the']:.4f}")
print(f"Wikipedia:\t{optimal_binary_length(model_wikipedia.dist)['the']:.4f}")
print(f"Code:\t\t{optimal_binary_length(model_code.dist)['the']:.4f}")

Optimal Encoding Length: "if"
Frankenstein:	9.0000
Wikipedia:	12.0000
Code:		7.0000

Optimal Encoding Length: "the"
Frankenstein:	5.0000
Wikipedia:	4.0000
Code:		6.0000


2. Write a function that returns an expected code length of a token sequence, given a probability distribution. [0.5 points]

In [109]:
#AGAIN, FORMULA IS IN SLIDE 35 of LECTURE 4

def expected_code_length(tokens: List[str], distribution: Dict[str, float]) -> float:
    """
    Calculate the expected code length for a given distribution.

    Args:
        distribution: A dictionary of token probabilities.
    Returns:
        float: The expected code length.
    """
    #expected code len = SUM-> (optimal bin len)* p(wi)
    #again optimal bin len is = ciel(-log2(p))

    expected_code_len = 0
    for token in tokens:
        p = distribution.get(token, 0)
        expected_code_len += p * ceil(-log2(p))
    
    return expected_code_len
        


In [110]:
expected_length_frankenstein = expected_code_length(tokens_frankenstein, model_frankenstein.dist)
expected_length_wikipedia = expected_code_length(tokens_wikipedia, model_wikipedia.dist)
expected_length_code = expected_code_length(tokens_code, model_code.dist)

print("Expected Code Length for Frankenstein:", expected_length_frankenstein)
print("Number of tokens in Frankenstein:", len(tokens_frankenstein))
print()
print("Expected Code Length for Wikipedia:", expected_length_wikipedia)
print("Number of tokens in Wikipedia:", len(tokens_wikipedia))
print()
print("Expected Code Length for Code:", expected_length_code)
print("Number of tokens in Code:", len(tokens_code))

Expected Code Length for Frankenstein: 4729.49492918808
Number of tokens in Frankenstein: 78094

Expected Code Length for Wikipedia: 78275.76804626374
Number of tokens in Wikipedia: 1800837

Expected Code Length for Code: 6446.640550953427
Number of tokens in Code: 448389


3. Consider four binary codes.

| word | $C_1(\text{word})$| $C_2(\text{word})$| $C_3(\text{word})$| $C_4(\text{word})$|
|:-----|:------------------|:------------------|:------------------|:------------------|
| the  |`100`              |`0`                |`0`                |`11`               |
| and  |`01`               |`10`               |`10`               |`110`              |
| of   |`110`              |`110`              |`110`              |`1011`             |
| and  |`1110`             |`1110`             |`111`              |`0`                |
| to   |`1111`             |`1111`             |`1111`             |`1101`             |

* Which of these codes are prefix-free? For other codes, explain why they are not. [0.5 points]

* Which of these codes satisfy Kraft's inequality? [0.5 points]

4. Prove mathematically that Kraft's inequality holds for all prefix-free binary codes. **HINT**: think about how many leaves there are at a binary tree's depth $l_n$. [2 points]

## Q3 Answers
We can tell if a code is prefix-free if it does occur in the beginning of any other code, for ex, Ci and Cj are prefixes of each other if Ci's code starts with C_j's or vice versa. By this logic we can tell: 
- for C1, it is prefix free because no code occurs in the beginning of any other
- for C2, it is prefix free because no code occurs in the beginning of any other
- for C3, it is NOT prefix free because the code for "and" occurs in the beginning of "to"
- for C4, it is NOT prefix free because the code foe "the" occurs in the beginning of "and"

<br/>
to check if they satisfy Kraft's inequality we need to check for 

$$\sum_{i} D^{-l_i} <= 1$$

where D is 2 (since binary) <br/>

For C1:
- 1/8 + 1/4 + 1/8 + 1/16 + 1/16 = 0.625 < 1 therefore it does satisfy the inequality 

For C2:
- 1/2 + 1/4 + 1/8 + 1/16 + 1/16 = 1<=1 therefore it does satisfy the inequality 

For C3:
- 1/2 + 1/4 + 1/8 + 1/8 + 1/16 = 1.06 <=1 therefore it does NOT satisfy the inequality 

For C4:
- 1/4 + 1/8 + 1/16 + 1/2 + 1/16 = 1<=1 therefore it does satisfy the inequality 


## Q4 Answers

Imagine a full binary tree where each node has 2 children. At depth $l$ there are at most $2^l$ nodes. So at depth of $l_{i}$ there are at most  $2^l_{i}$ leaf positions. <br/>

Once a codeword is placed at a leaf, none of its descendants can be used for any other codeword (to preserve prefix-freeness). <br/>
Hence, placing a codeword at depth $l_i$ blocks off all $2^{L-l_i}$ leaves at the deeper level $L$ <br/>

Now we can choose an $L$ that $L>= max(l_i)$. Imagine  extending all codewords to depth $L$. Each codeword of length $l_i$ has $2^{L-l_i}$ descendants at depth L. We can say these are the leaves it claims. <br/>

Therefore, the sum of claimed leaves is $\sum_{i=1} 2^{L-l_i}$ <br/>
Since all code words must occupy non-overlapping set of leaves, the total number of claimed leaves cannot exceed the total number of leaves at depth L or rather $2^L$ <br/>
So, $\sum_{i=1} 2^{L-l_i} <= 2^L$ <br/>

Divide both sides by $2^L$ and we get: <br/>

$\sum_{i=1} 2^{-l_i} <= 1$





